[14 数值稳定性 + 模型初始化和激活函数【动手学深度学习v2】](https://www.bilibili.com/video/BV1u64y1i75a?spm_id_from=333.999.0.0)

### 神经网络的梯度
- 梯度输出 = 一层输入*二层输出*。。。。。损失函数
- $$y = loss * f_d* ... f_1(x)$$
- 那么梯度变化等于每层的梯度变化的乘积
- $$\frac{\partial \iota}{\partial W_t} = \frac{\partial \iota}{\partial h^d}\frac{\partial h^d}{\partial h^{d-1}} ...\frac{\partial h^{t+1}}{\partial h^t}\frac{\partial h^t}{\partial W^t}$$
- 一共有d-t次矩阵乘法
- 容易导致：一、梯度爆炸 二、梯度消失（就是容易过大或者过小）也就是会带来数值的不稳定性
- 公式推导见pdf

### 梯度爆炸问题
- 值超出值域
  - 对于16位浮点数尤为严重（数值区间为6e-5至6e4）
- 对学习率敏感
  - 学习率过大->大参数值->更大的梯度
  - 学习率过小->训练无法进展
  - 我们可能需要在训练过程中不断调整学习率 

- 假设
  - $w^t_{i,j}$为初始权重，那么这里先要求权重能做到$E[w^t_{i,j}]=0,Var[w^t_{i,j}]=\gamma$
  - $h^{t-1}_i与w^t_{i,j}$相互独立
  - 这里首先记得公式$E[\sum x_iy_j]=\sum E[x]E[y]$

- 首先我们看正向方差 
  - 这里先不考虑激活函数那么有 $h^t=W^th^{t-1}$
    - $Var[h_i^t]=E[h_i^t]^2-[Eh^t_i]^2=E[(\sum w_{i,j}^th^{t-1}_j)^2]$
    - 方差的期望=平方的期望-期望的平方，后者为0
    - $=E[\sum (w^t_{i,j})^2(h^{t-1}_j)^2+\sum_{j\neq k} w^t_{i,j}w^t_{i,k}h^{t-1}_jh^{t-1}_k]$
    - 同样此项的后半部分同样拆开为多个期望相乘，为0舍去
    - $= \sum_j E[(w^t_{i,j})^2]E[(h^{t-1}_j)^2]$
    - 期望为0的情况下，平方的期望就是方差
    - $= \sum_j Var[w^t_{i,j}]Var[h^{t-1}_j]$
    - $= n_{t-1}\gamma_t Var[h^{t-1}_j]$
    - $h^{t-1}_j$如果要求正向传播的方差不变，即$Var[h_i^t]=Var[h^{t-1}_j]$，那么则有$n_{t-1}\gamma_t=1$
    - 注：这里n为对应层数的维数，即该层的神经元数量，$\gamma$则为我们需要设置的方差值

- 接下来是反向传播了
  - 正向传播是传递函数本身，反向传播传递的是梯度
  - 反向传播求的是求损失函数的梯度，实际计算采用链式法则来实现
  - 这里只考虑单层的情况，所以不考虑链式法则的影响
  - 同样对于单层的感知机$h^t=W^th^{t-1}$
  - 满足：$\frac{\partial \iota}{\partial h^{t-1}} = \frac{\partial \iota}{\partial h^{t}}W^t$ ; $E[\frac{\partial \iota}{\partial h^{t-1}}]=0$
  - 通过采用与之前正向传播同样的方式，可以计算出
  - $Var[\frac{\partial \iota}{\partial h^{t-1}}]=n_t \gamma_t Var[\frac{\partial \iota}{\partial h^{t}}]$
  - 那么可以得出$n_t \gamma_t=1$

- Xavier初始化
  - 实际中$n_t \gamma_t=1 ; n_{t-1} \gamma_t=1$难以同时满足，因为这意味着每一层的维数，即神经元的数量保持不变
  - 既然无法同时满足，那么就采取折中方案：$\gamma_t (n_{t-1}+n_t)/2 =1$
  - 可以采用两种方式进行权重分配
    - 1、正态分布，$N(0,\sqrt \frac{2}{n_{t-1}+n_t})$
    - 2、均匀分布，$U(-\sqrt \frac{6}{n_{t-1}+n_t},\sqrt \frac{6}{n_{t-1}+n_t})$
    - 两种分布方差期望均相等

- 如果考虑激活函数，会如何变化？
- 如果激活函数为线性函数
  - $\sigma (x) = \alpha x + \beta$
  - 有$Var[h^t_i] = \alpha^2Var[W^th^{t-1}_i]$
  - 反向亦是如此
  - 这里可以推导出$\alpha = 1 ; \beta = 0$
  - 那么理论上$\sigma (x) = x$是最好的。。。很无语的结果

- 对激活函数采用泰勒展开
  - $sigmoid(x) = \frac {1}{2}+\frac {x}{4}-\frac {x^3}{48}$
  - $tanh(x) = x - \frac {x^3}{3}$
  - relu(x) = x for  x ≥0
  - 这里可看到，tanh和relu近似接近于f(x) = x
  - 或者将4sigmoid(x)-2作为激活函数

- 我们得出了什么结论？
  - 一、每一层的权重分配可以遵循$N(0,\sqrt \frac{2}{n_{t-1}+n_t})$分布
  - 二、尽量选择relu函数作为激活函数，皮实耐用